In [1]:
import os, sys, inspect
import csv

import math
import numpy as np
import pandas as pd


import matplotlib
from matplotlib import pyplot as plt

import collections
import time
import datetime

import re

import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException

working_dir = os.path.dirname(
    os.path.abspath(
        inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(working_dir)
sys.path.insert(0, parent_dir)

from scripts_python import scraper as scr

%load_ext autoreload
%autoreload 2

1. Chrome - Help - About Google Chrome
2. check version; e.g. `Version 79.0.3945.117 (Official Build) (64-bit)`
3. [download relevant ChromeDriver](https://chromedriver.chromium.org/downloads)

In [4]:
with open('../ignorabilia/glassdoor_scraper.csv', mode='r') as f_in:
    credentials = {k: v for k, v in csv.reader(f_in)}

In [5]:
browser = scr.get_web_browser()
browser.implicitly_wait(10)
scr.load_main_page(browser=browser)

scr.accept_cookies(browser=browser)

scr.sign_in(
    browser=browser,
    e_mail=credentials['e_mail'],
    password=credentials['password'])

https://www.glassdoor.co.uk


In [6]:
scr.do_search(
        browser=browser,
        keywords='data scientist',
        location='Paris (France)')

In [7]:
scr.set_filters(
        browser=browser,
        jobtype='fulltime',
        post_age='7',
        radius='5',
        )

In [8]:
search_results = []
search_date = datetime.datetime.now()

In [9]:
browser.implicitly_wait(10)
element = browser.find_element_by_xpath(
    f"//div[@id='PageContent']"
    f"//div[@id='JobResults']"
    f"//p[@class='jobsCount']"
    )

n_jobs = int(element.text.split()[0])
n_pages = math.ceil(346 / 30)

In [10]:
element.text

'397 Jobs'

In [11]:
for i in range(n_pages-8):
    search_results = browser.find_elements_by_xpath(
        f"//div[@id='PageContent']"
        f"//div[@id='JobResults']"
        f"//ul[@class='jlGrid hover']"
        f"//li[@class='jl']")
    
    for result in search_results[:5]:
#         result = {}
        result.click()
        browser.implicitly_wait(5)
        
        name, rating = scr.get_company_name_and_rating(browser=browser)
        job_title, job_description = scr.get_job_title_and_description(browser=browser)
        size, url = scr.get_company_size_and_url(browser=browser)
        search_results.append({
            'search_date': search_date,
            'company_name': name,
            'company_rating': rating,
            'company_size': size,
            'company_url': url,
            'job_title': job_title,
            'job_description': job_description})
        
    scr.switch_to_next_page(browser=browser)
    

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=79.0.3945.117)
